# Flight Delay prediction

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn,optim
import torch.nn.functional as F
import datetime
from torch.utils import data
from random import sample,seed

## Importing Flight Data

In [2]:
flights = pd.read_csv("./data/flights.csv",low_memory=False)
airlines = pd.read_csv("./data/airlines.csv")
airports = pd.read_csv("./data/airports.csv")

## Preprocessing of the data

In [3]:
airports = {ch:i for i,ch in enumerate(flights.ORIGIN_AIRPORT.unique())}
airlines = {ch:i for i,ch in enumerate(airlines.IATA_CODE)}
airports['10666'] =len(airports)

In [4]:
# one hot encoder for day of the week
def one_hot_encode(size,val):
    a = np.zeros((size,),dtype=int)
    a[(val-1)] = 1
    return a

In [5]:
# Date and time preprocess copied from
# link - https://www.kaggle.com/fabiendaniel/predicting-flight-delays-tutorial/data
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [6]:
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

In [7]:
# REFORMATING THE DATE VARIABLES
#__________________________________________________________________________________
flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_heure) #create_flight_time(flights, 'SCHEDULED_DEPARTURE')
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)
#__________________________________________________________________________
flights.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,00:05:00,04:30:00,23:54:00,04:08:00,-11.0,-22.0
1,00:10:00,07:50:00,00:02:00,07:41:00,-8.0,-9.0
2,00:20:00,08:06:00,00:18:00,08:11:00,-2.0,5.0
3,00:20:00,08:05:00,00:15:00,07:56:00,-5.0,-9.0
4,00:25:00,03:20:00,00:24:00,02:59:00,-1.0,-21.0
5,00:25:00,06:02:00,00:20:00,06:10:00,-5.0,8.0


In [8]:
# DROPING IRRELEVANT COLUMNS
#_____________________________________________________________________________________
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'DAY','DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
flights.drop(variables_to_remove, axis = 1, inplace = True)
flights = flights[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME', 'ELAPSED_TIME','MONTH','DAY_OF_WEEK']]
flights[:5]

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,SCHEDULED_TIME,ELAPSED_TIME,MONTH,DAY_OF_WEEK
0,AS,ANC,SEA,00:05:00,23:54:00,-11.0,04:30:00,04:08:00,-22.0,205.0,194.0,1,4
1,AA,LAX,PBI,00:10:00,00:02:00,-8.0,07:50:00,07:41:00,-9.0,280.0,279.0,1,4
2,US,SFO,CLT,00:20:00,00:18:00,-2.0,08:06:00,08:11:00,5.0,286.0,293.0,1,4
3,AA,LAX,MIA,00:20:00,00:15:00,-5.0,08:05:00,07:56:00,-9.0,285.0,281.0,1,4
4,AS,SEA,ANC,00:25:00,00:24:00,-1.0,03:20:00,02:59:00,-21.0,235.0,215.0,1,4


In [9]:
# PROCESSING THE CATEGORICAL DATA
#___________________________________________________________________
flights['AIRLINE'] = flights['AIRLINE'].apply(lambda x: airlines[x])
flights['ORGIN_AIRPORT_V'] = flights['ORIGIN_AIRPORT'].apply(lambda x: airports[x])
flights['DESTINATION__AIRPORT_V'] = flights['DESTINATION_AIRPORT'].apply(lambda x: airports[x])
flights['DAY_OF_WEEK'] = flights['DAY_OF_WEEK'].apply(lambda x: one_hot_encode(7,x))
flights = flights[['AIRLINE','ORGIN_AIRPORT_V',"DESTINATION__AIRPORT_V",'ARRIVAL_DELAY','DAY_OF_WEEK','DEPARTURE_DELAY']]
flights.head()

,AIRLINE,ORGIN_AIRPORT_V,DESTINATION__AIRPORT_V,ARRIVAL_DELAY,DAY_OF_WEEK,DEPARTURE_DELAY
0,6,0,3,-22.0,"[0, 0, 0, 1, 0, 0, 0]",-11.0
1,1,1,77,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-8.0
2,2,2,97,5.0,"[0, 0, 0, 1, 0, 0, 0]",-2.0
3,1,1,84,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-5.0
4,6,3,0,-21.0,"[0, 0, 0, 1, 0, 0, 0]",-1.0


In [10]:
# CLEAN UP
#______________________________
flights.dropna(inplace =True)
flights = flights.reset_index()
flights = flights[['AIRLINE','ORGIN_AIRPORT_V',"DESTINATION__AIRPORT_V",'ARRIVAL_DELAY','DAY_OF_WEEK','DEPARTURE_DELAY']]
flights.head()

,AIRLINE,ORGIN_AIRPORT_V,DESTINATION__AIRPORT_V,ARRIVAL_DELAY,DAY_OF_WEEK,DEPARTURE_DELAY
0,6,0,3,-22.0,"[0, 0, 0, 1, 0, 0, 0]",-11.0
1,1,1,77,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-8.0
2,2,2,97,5.0,"[0, 0, 0, 1, 0, 0, 0]",-2.0
3,1,1,84,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-5.0
4,6,3,0,-21.0,"[0, 0, 0, 1, 0, 0, 0]",-1.0


In [11]:
# MAKING THE SEQUENCES OF INPUT DATA
#________________________________________________________________
spequence_length = 1000
output_length = 1
start = 0
data_batch = {}
decoder_input ={}
labels = {}
for i,x in enumerate(range(0,len(flights),spequence_length)):
#     print('id_{}:{}'.format(i,x))
#     y = x+spequence_length
    data_batch['id_{}'.format(i)] = [i for i in range(start,x)]
    air = flights.loc[x:x+output_length-1,:'DAY_OF_WEEK']
    f = flights.loc[x:x+output_length-1,'ARRIVAL_DELAY']
    decoder_input['id_{}'.format(i)] = air
    labels['id_{}'.format(i)] = f.to_list()
    start = x

In [12]:
# DATA GENERATOR CLASS FOR EASY ACCESS AND PARALLELIZATION
#___________________________________________________________
class DataGen(data.Dataset):
    def __init__(self,list_id,labels):
        self.list_id = list_id
        self.labels = labels
    def __len__(self):
        return len(self.list_id)
    def __getitem__(self,index):
        ID = self.list_id[index]
        ind = data_batch[ID]
        X = flights.loc[ind,:'DAY_OF_WEEK']
        d_inp = decoder_input[ID]
        d_inp = {'DAY_OF_WEEK':torch.tensor(d_inp.DAY_OF_WEEK.to_list(),dtype=torch.float),
                'AIRLINE': torch.tensor(d_inp.AIRLINE.to_list(),dtype=torch.float),
                'ORGIN_AIRPORT_V': torch.tensor(d_inp.ORGIN_AIRPORT_V.to_list(),dtype=torch.float),
                  'DESTINATION__AIRPORT_V': torch.tensor(d_inp.DESTINATION__AIRPORT_V.to_list(),dtype=torch.float)}
        y = torch.tensor(labels[ID],dtype=torch.float)
        return d_inp,y,{'DAY_OF_WEEK':torch.tensor(X.DAY_OF_WEEK.to_list(),dtype=torch.float),
                  'AIRLINE': torch.tensor(X.AIRLINE.to_list(),dtype=torch.float),
                  'ARRIVAL_DELAY': torch.tensor(X.ARRIVAL_DELAY.to_list(),dtype=torch.float),
                  'ORGIN_AIRPORT_V': torch.tensor(X.ORGIN_AIRPORT_V.to_list(),dtype=torch.float),
                  'DESTINATION__AIRPORT_V': torch.tensor(X.DESTINATION__AIRPORT_V.to_list(),dtype=torch.float)}

In [13]:
# PARAMETER FOR THE GENERATOR
#_____________________________
params = {'batch_size': 32,
          'shuffle': False,
          'num_workers': 6}

In [14]:
# SPLITING UP THE DATA INTO TRAINING AND VALIDATION
#_____________________________________________________
seed(1234)
data_ids=list(data_batch.keys())
data_ids.pop(0)
length = int(len(data_ids)*0.75)
training_ids = sample(data_ids,length)
validation_ids = []
for i in list(data_batch.keys()):
    if i not in training_ids:
        validation_ids.append(i)

In [15]:
#CREATING DATALOADERS
#_______________________________________________________________
training_set = DataGen(training_ids, labels)
training_generator = data.DataLoader(training_set, **params)
validation_ids.pop(0)
validation_set = DataGen(validation_ids, labels)
validation_generator = data.DataLoader(validation_set,batch_size=2,shuffle=False)

# MODEL DEFINITION

In [16]:
cuda =torch.cuda.is_available()
device = torch.device('cuda:0'if torch.cuda.is_available() else "cpu")
cuda

True

In [20]:
# MODEL DEFINITION
#_________________
class model(nn.Module):
    
    def __init__(self,airline_embed_size=4,airport_embed_size=8,hidden_size=83,no_of_layers=2,dropout=0.25,linear_size=512):
        super(model,self).__init__()
        
        # DEFINING THE VARIABLES AND PARAMETERS
#         self.airline_embed_size = airline_embed_size
#         self.airport_embed_size = airport_embed_size
        self.hidden_size = hidden_size
        self.no_of_layers =no_of_layers
        self.hidden_size = (1+2+7) + 2048*(1+2+7+1) #(airport_embed_size+airline_embed_size+7) + 2048*(airport_embed_size+airline_embed_size+7+1)
        
        # MODEL ELEMENTS
#         self.airline_embed = nn.Embedding(len(airlines),airline_embed_size)
#         self.airport_embed = nn.Embedding(len(airports),airport_embed_size) 
        self.conv1 = nn.Conv1d(1000,2048,kernel_size=2,dilation=2,padding=1)
#         self.conv2 = nn.Conv1d(2048,1024,kernel_size=2,stride=1,dilation=2,padding=1)
#         self.conv3 = nn.Conv1d(1024,2048,kernel_size=2,stride=1,dilation=2,padding=1)
#         self.conv4 = nn.Conv1d(2048,2048,1)
#         self.conv5 = nn.Conv1d(2048,3072,1)
        # HIDDEN SIZE IS CALCULATED BASED ON DOWN SIZING THROUGH CONVOLUTION
        self.bnorm = nn.BatchNorm1d(num_features=linear_size)
        self.linear_1 = nn.Linear(self.hidden_size,linear_size)
        self.linear_2 = nn.Linear(linear_size,linear_size)
        self.out = nn.Linear(linear_size,1)
        self.drop = nn.Dropout(p=dropout)
        
    def forward(self,x,decode):
#         print(x['AIRLINE'].is_cuda)
        decode['DAY_OF_WEEK'] = decode['DAY_OF_WEEK'].to(device)
        decode['AIRLINE'] = decode['AIRLINE'].to(device)
        decode['DESTINATION__AIRPORT_V'] = decode['DESTINATION__AIRPORT_V'].to(device)
        decode['ORGIN_AIRPORT_V'] = decode['ORGIN_AIRPORT_V'].to(device)
        x['DAY_OF_WEEK'] = x['DAY_OF_WEEK'].to(device)
        x['AIRLINE'] = x['AIRLINE'].to(device)
        x['ARRIVAL_DELAY'] = x['ARRIVAL_DELAY'].to(device)
        x['ARRIVAL_DELAY'] = x['ARRIVAL_DELAY'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['AIRLINE']=x['AIRLINE'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['DESTINATION__AIRPORT_V'] =  x['DESTINATION__AIRPORT_V'].to(device)
        x['DESTINATION__AIRPORT_V'] =x['DESTINATION__AIRPORT_V'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['ORGIN_AIRPORT_V'] = x['ORGIN_AIRPORT_V'].to(device)
        x['ORGIN_AIRPORT_V'] = x['ORGIN_AIRPORT_V'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        input_ =torch.cat((x['DAY_OF_WEEK'],x['AIRLINE'],x['DESTINATION__AIRPORT_V'],x['ORGIN_AIRPORT_V'],x['ARRIVAL_DELAY']),2)
        # CONCATINATED DAY OF THE WEEK, DELAY, AIRLINE AND DESTINATION-ORIGIN(GRAPH REP)
        
#         decode['AIRLINE'] = self.airline_embed(decode['AIRLINE'])
#         decode['DESTINATION__AIRPORT_V'] = self.airport_embed(decode['DESTINATION__AIRPORT_V'])
#         decode['ORGIN_AIRPORT_V'] = self.airport_embed(decode['ORGIN_AIRPORT_V'])

        out = F.relu(self.conv1(input_))
#         out = F.relu(self.conv2(out))
#         out = F.relu(self.conv3(out))
#         out = F.relu(self.conv4(out))
#         out = F.relu(self.conv5(out))
        
        out = out.view(out.shape[0],-1)
        decode['ORGIN_AIRPORT_V'] = decode['ORGIN_AIRPORT_V'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['AIRLINE'] = decode['AIRLINE'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['DESTINATION__AIRPORT_V'] = decode['DESTINATION__AIRPORT_V'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['DAY_OF_WEEK'] = decode['DAY_OF_WEEK'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        out = torch.cat((out,decode['DAY_OF_WEEK'],decode['DESTINATION__AIRPORT_V'],decode['ORGIN_AIRPORT_V'],decode['AIRLINE']),1)
        
        out = self.drop(F.relu(self.linear_1(out)))
        out = self.drop(F.relu(self.linear_2(out)))
        out = self.out(out)
        
        return out
    

In [21]:
#LOSS AND OPTIMISATION FUNCTIONS
conv = model()
criterian = nn.MSELoss()  #nn.SmoothL1Loss() 
# It uses a squared term if the absolute error falls below 1 and an absolute term otherwise.
optimizer = optim.Adam(conv.parameters(),lr =0.001)

# Training

In [22]:
epoch = 25
conv.to(device)
g_loss = float("inf")
for e in range(0,epoch):
    running_loss =0
    conv.train()
    count =0
    for decode,y,x in training_generator: 
#         decode = decode.to(device)
        y = y.to(device)
        out = conv(x,decode)
        loss = criterian(out,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss +=loss.item()
        count =count+1
    else:
        if((e+1)%5==0):
            val_loss =0 
            conv.eval()
            count_val = 0
            for decode,y,x in validation_generator:
#                 decode = decode.to(device)
                y = y.to(device)
                out = conv(x,decode)
                loss = criterian(out,y)
                val_loss +=loss.item()
                count_val+=1
            print(f'The Training loss = {running_loss/count} : Validation loss is {val_loss/count_val}')
            if(val_loss < g_loss):
                print("Saving model....")
                torch.save(conv.state_dict(),"./conv.pt")
                g_loss = val_loss


The Training loss = 1060.4131681527665 : Validation loss is 2964.699688880427
Saving model....
The Training loss = 819.2649052178682 : Validation loss is 2112.221067659109
Saving model....
The Training loss = 772.4208375162153 : Validation loss is 1455.632027367737
Saving model....
The Training loss = 689.0698765996676 : Validation loss is 2857.9961993825186
The Training loss = 650.6036478868172 : Validation loss is 2682.8112780726874


# Testing

In [23]:
# spequence_length = 1000
conv.load_state_dict(torch.load("./conv.pt"))
conv.eval()
# count_in =0
# count = 0
# for i,ind in enumerate(range((len(flights)-1000),len(flights))):
#     if((i+1)%10000 == 0):
#         print(i)
#     X = flights.loc[ind-spequence_length:ind-1,:'DAY_OF_WEEK']
#     x = {'DAY_OF_WEEK':torch.tensor(X.DAY_OF_WEEK.to_list(),dtype=torch.float).reshape(1,spequence_length,-1),
#         'AIRLINE': torch.tensor(X.AIRLINE.to_list(),dtype=torch.long),
#          'ARRIVAL_DELAY': torch.tensor(X.ARRIVAL_DELAY.to_list(),dtype=torch.float),
#         'ORGIN_AIRPORT_V': torch.tensor(X.ORGIN_AIRPORT_V.to_list(),dtype=torch.long),
#         'DESTINATION__AIRPORT_V': torch.tensor(X.DESTINATION__AIRPORT_V.to_list(),dtype=torch.long)}
#     d = flights.loc[ind,'AIRLINE']
#     test_decode = torch.tensor([d],dtype=torch.long)
#     decode = decode.to(device)
#     out = conv(x,decode)
#     if(abs(out) <= 10):
#         count_in +=1
#     count+=1
for decode,y,x in validation_generator:
#             decode = decode.to(device)
            out = conv(x,decode)
            out = out
#             for i in out:
#                 if(abs(i) <= 10):
#                     count_in +=1
#                 count+=1
            print('The predicted value is {} and the actual {}'.format(out,y))

The predicted value is tensor([[-0.3433],
        [-0.6842]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 10.],
        [-19.]])
The predicted value is tensor([[-0.5097],
        [-0.5326]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -9.],
        [-19.]])
The predicted value is tensor([[ 1.4774],
        [-0.1846]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-25.],
        [ 12.]])
The predicted value is tensor([[ 0.5588],
        [-0.1226]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 5.],
        [18.]])
The predicted value is tensor([[ 0.6847],
        [59.1428]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-12.],
        [-10.]])
The predicted value is tensor([[53.4877],
        [-0.2857]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[18.],
        [ 0.]])
The predicted value is tensor([[179.2211],
        [ 62.7536]], device='cuda:0', grad_fn=<Ad

The predicted value is tensor([[-0.6384],
        [-0.5235]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-36.],
        [ 37.]])
The predicted value is tensor([[ 7.9955],
        [-0.1569]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -7.],
        [-14.]])
The predicted value is tensor([[-0.6284],
        [-0.5327]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 36.],
        [112.]])
The predicted value is tensor([[16.2790],
        [ 2.4043]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 24.],
        [-10.]])
The predicted value is tensor([[-0.1682],
        [-0.3957]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 7.],
        [-3.]])
The predicted value is tensor([[-0.4710],
        [-0.3821]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-24.],
        [ 85.]])
The predicted value is tensor([[ 7.3044],
        [-0.4944]], device='cuda:0', grad_fn=<Ad

The predicted value is tensor([[41.2452],
        [-0.1904]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 5.],
        [-8.]])
The predicted value is tensor([[52.3812],
        [-0.6404]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-9.],
        [ 6.]])
The predicted value is tensor([[ 9.8067],
        [38.5422]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-6.],
        [-2.]])
The predicted value is tensor([[4.5347],
        [1.5713]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-2.],
        [92.]])
The predicted value is tensor([[31.4529],
        [-0.0406]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 77.],
        [103.]])
The predicted value is tensor([[-0.2519],
        [-0.3199]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[6.],
        [3.]])
The predicted value is tensor([[19.0317],
        [ 1.1298]], device='cuda:0', grad_fn=<AddmmBackward>

The predicted value is tensor([[-0.3653],
        [-0.2388]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-11.],
        [ -1.]])
The predicted value is tensor([[-0.4432],
        [-0.4254]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-13.],
        [-13.]])
The predicted value is tensor([[-0.4392],
        [ 3.3327]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 49.],
        [-21.]])
The predicted value is tensor([[-0.4175],
        [-0.3500]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -9.],
        [-16.]])
The predicted value is tensor([[-0.3241],
        [-0.4024]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[14.],
        [19.]])
The predicted value is tensor([[-0.4317],
        [-0.6807]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -3.],
        [-27.]])
The predicted value is tensor([[-0.2496],
        [-0.3725]], device='cuda:0', grad_fn=<Ad

The predicted value is tensor([[-0.4567],
        [-0.3317]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-15.],
        [ 84.]])
The predicted value is tensor([[-0.4228],
        [-0.1754]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -2.],
        [-14.]])
The predicted value is tensor([[-0.2060],
        [-0.0392]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[  1.],
        [-11.]])
The predicted value is tensor([[-0.3201],
        [ 0.2267]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -9.],
        [-19.]])
The predicted value is tensor([[8.1445],
        [0.0949]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-11.],
        [  4.]])
The predicted value is tensor([[-0.0662],
        [-0.2520]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-3.],
        [-6.]])
The predicted value is tensor([[-0.4593],
        [-0.5152]], device='cuda:0', grad_fn=<Addm

...

The predicted value is tensor([[-0.4066],
        [-0.0200]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-5.],
        [ 4.]])
The predicted value is tensor([[-0.3696],
        [-0.4598]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[14.],
        [-1.]])
The predicted value is tensor([[-0.6319],
        [-0.4398]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-24.],
        [-15.]])
The predicted value is tensor([[-0.3847],
        [-0.1024]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[  0.],
        [-12.]])
The predicted value is tensor([[-0.3880],
        [-0.5135]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -9.],
        [-19.]])
The predicted value is tensor([[-0.2792],
        [-0.1674]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -4.],
        [-12.]])
The predicted value is tensor([[-0.5391],
        [-0.4415]], device='cuda:0', grad_fn=<Addm

The predicted value is tensor([[-0.3445],
        [-0.4972]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 0.],
        [60.]])
The predicted value is tensor([[-0.2344],
        [-0.3470]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-15.],
        [ -6.]])
The predicted value is tensor([[ 6.5520],
        [-0.7995]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-24.],
        [-15.]])
The predicted value is tensor([[-0.3534],
        [-0.2243]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-13.],
        [ 86.]])
The predicted value is tensor([[-0.2484],
        [-0.5925]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-28.],
        [ 16.]])
The predicted value is tensor([[-0.4668],
        [-0.4128]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-5.],
        [31.]])
The predicted value is tensor([[-0.4610],
        [-0.4247]], device='cuda:0', grad_fn=<Addm

The predicted value is tensor([[-2.3830],
        [-2.7925]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[  5.],
        [-28.]])
The predicted value is tensor([[-2.5522],
        [-2.3077]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[103.],
        [-13.]])
The predicted value is tensor([[-2.6618],
        [-2.3739]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-13.],
        [-17.]])
The predicted value is tensor([[-2.6334],
        [-2.5835]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[19.],
        [-6.]])
The predicted value is tensor([[-2.4564],
        [-2.8153]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-2.],
        [ 5.]])
The predicted value is tensor([[-2.5070],
        [-2.7755]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-10.],
        [ -5.]])
The predicted value is tensor([[-2.7177],
        [-2.4310]], device='cuda:0', grad_fn=<Addm

The predicted value is tensor([[-0.5930],
        [-0.3591]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -2.],
        [-10.]])
The predicted value is tensor([[ 3.1246],
        [-0.5121]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 12.],
        [-37.]])
The predicted value is tensor([[-0.3234],
        [37.2082]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-23.],
        [-23.]])
The predicted value is tensor([[-0.4724],
        [-0.3683]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-15.],
        [ -6.]])
The predicted value is tensor([[-0.3566],
        [-0.2551]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-26.],
        [  3.]])
The predicted value is tensor([[-0.2604],
        [-0.2126]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[1.],
        [3.]])
The predicted value is tensor([[-0.5258],
        [-0.2770]], device='cuda:0', grad_fn=<Addm

The predicted value is tensor([[-0.1851],
        [-0.1381]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -7.],
        [-16.]])
The predicted value is tensor([[-0.2580],
        [-0.2672]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-16.],
        [ -3.]])
The predicted value is tensor([[-0.2469],
        [ 4.7795]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-9.],
        [-7.]])
The predicted value is tensor([[-0.2605],
        [-0.3269]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[16.],
        [ 4.]])
The predicted value is tensor([[-0.4506],
        [-0.1595]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-6.],
        [17.]])
The predicted value is tensor([[ 0.2456],
        [-0.1675]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[20.],
        [-5.]])
The predicted value is tensor([[ 4.8522],
        [48.5562]], device='cuda:0', grad_fn=<AddmmBac

In [ ]:
# (count_in/count)*100

In [ ]:
for i,ind in enumerate(range(len(flights)-100,len(flights))):
    if((i+1)%10 == 0):
        print(i)
    X = flights.loc[ind-spequence_length:ind-1,:'DAY_OF_WEEK']
    x = {'DAY_OF_WEEK':torch.tensor(X.DAY_OF_WEEK.to_list(),dtype=torch.float).reshape(1,spequence_length,-1),
        'AIRLINE': torch.tensor(X.AIRLINE.to_list(),dtype=torch.long),
         'ARRIVAL_DELAY': torch.tensor(X.ARRIVAL_DELAY.to_list(),dtype=torch.float),
        'ORGIN_AIRPORT_V': torch.tensor(X.ORGIN_AIRPORT_V.to_list(),dtype=torch.long),
        'DESTINATION__AIRPORT_V': torch.tensor(X.DESTINATION__AIRPORT_V.to_list(),dtype=torch.long)}
    d = flights.loc[ind,'AIRLINE']
    test_decode = torch.tensor([d],dtype=torch.long)
    decode = decode.to(device)
    out = conv(x,decode)
    a = flights.loc[ind,'ARRIVAL_DELAY']
    print('The predicted value is {} and the actual {}'.format(out,a))